In [1]:
from lark import Lark
import json

Grammar = """
?json_path_expression : path

path :    "$"                                     -> root
    | "@"                                         -> current
    | path "." CNAME                              -> dot
    | path "." CNAME "()"                         -> method
    | path "?" "(" expression ")"                 -> condition
    | path "[" index "]"                          -> subscript

!index : "*"
    | SIGNED_INT

?expression : or_expression

?or_expression : and_expression
    | or_expression "||" and_expression
    
?and_expression : term_expression
    | and_expression "&&" term_expression

?term_expression : cmp_expression
    | "(" expression ")"

cmp_expression : path CMPOP value

value : STRING
    | SIGNED_INT
    | FLOAT
    | BOOL

CMPOP:  "<" "="? | "!"? "=" "="? | "<" ">" | "!"? "~" "*"? | ">" "="? | "like_regex"

BOOL: "true"i | "false"i
STRING : /("(?!"").*?(?<!\\\\)(\\\\\\\\)*?"|'(?!'').*?(?<!\\\\)(\\\\\\\\)*?')/i
CNAME: ("_"|LETTER) ("_"|LETTER|DIGIT)*

%import common.LETTER
%import common.FLOAT
%import common.DIGIT
%import common.SIGNED_INT
%import common.SIGNED_FLOAT
%import common.WS
%ignore WS


"""

parser = Lark(Grammar, start="json_path_expression")


data = json.loads("""
{
  "track": {
    "segments": [
      {
        "location":   [ 47.763, 13.4034 ],
        "start time": "2018-10-14 10:05:14",
        "HR": 73
      },
      {
        "location":   [ 47.706, 13.2635 ],
        "start time": "2018-10-14 10:39:21",
        "HR": 135
      }
    ]
  }
}""")


In [33]:
from trees import Ascender, Converter, Node, LarkToNodes

class PathConverter(LarkToNodes):
    
    def dot(self, args):
        return Node("dot", [args[0]], member=args[1].value)
    
    def index(self, args):
        arg = args[0]
        if arg.value == "*":
            return "*"
        else:
            return int(arg.value)
    
    def subscript(self, args):
        node, index = args
        return Node("subscript", [node], index=index)
    
    def value(self, args):
        v = args[0]
        if v.type == "SIGNED_INT":
            return int(v.value)
        elif v.type == "FLOAT":
            return float(v.value)
        elif v.type == "STRING":
            return v.value[1:-1]
        elif v.type == "BOOL":
            return v.value.lower() == "true"
        
    def cmp_expression(self, args):
        return Node("cmp_expression", [args[0], args[2]], op=args[1].value)
            

    def __default__(self, typ, children, meta):
        return Node(typ, children, _meta=meta)
    



In [34]:
class PathEvaluator(Ascender):
    
    # evaluates the path expression to a list of nodes
    
    def evaluate(self, path_expression, root, current=None):
        #print("PathEvaluator.evaluate: path:", path_expression.pretty())
        #print("                        current:", current)
        self.Root = root
        self.Current = current
        out = self.walk(path_expression, debug=False)
        assert out.T == "values"
        return out
        
    def root(self, t):
        #print("root")
        out = Node("values", values=[self.Root])
        #print("root: returning:", out.pretty())
        return out
    
    def current(self, t):
        if self.Current is None:
            return t.clone()
        else:
            return Node("values", values=self.Current)
    
    def dot(self, t, node, member=None):
        if node.T != "values":
            return t.clone()   # return the node without changes
        values = node["values"]
        valid = [v for v in values if isinstance(v, dict) and member in v]
        return Node("values", values=[n[member] for n in valid])

    def subscript(self, t, node, index=None):
        if node.T != "values":
            return t.clone()
        out = []
        for n in node["values"]:
            if index == "*":
                if isinstance(n, dict):
                    out.extend(list(n.values()))
                elif isinstance(n, list):
                    out.extend(n)
            else:
                # assume index is integer
                if isinstance(n, dict) and index in n \
                    or isinstance(n, list):
                        try:   out.append(n[index])
                        except (IndexError, KeyError):
                            pass
        return Node("values", values=out)

    def condition(self, t, node, expression):
        if node.T != "values":
            return t.clone()   # filter node without change
        values = node["values"]
        print("PathEvaluator.condition: input values:", values)
        exp_evaluator = ConditionEvaluator(expression)
        out = []
        for v in values:
            print("   evaliating value:", v)
            passed = not not exp_evaluator.evaluate(v)
            print("   evaluated:", passed)
            if passed:
                out.append(v)
        return Node("values", values=out)

In [40]:
class ConditionEvaluator(Ascender):
    
    def __init__(self, expression):
        self.ExpressionTree = expression
        self.Current = None   # the value of "@"
    
    def evaluate(self, value):
        print("ConditionEvaluator.evaluate(): tree:", id(self.ExpressionTree),"\n", self.ExpressionTree.pretty(indent="    "))
        self.Current = value
        result = self.walk(self.ExpressionTree)
        print("ConditionEvaluator.evaluate(): result:\n", result.pretty(indent="    "))
        return result
    
    def current(self, t):
        print("ConditionEvaluator.current(): returning values:", [self.Current])
        return Node("values", values=[self.Current])
    
    def or_expression(self, t, left, right):
        return left or right
    
    def and_expression(self, t, left, right):
        return left and right
    
    def value(self, v):
        if v.T == "SIGNED_INT":
            return int(v.V)
        elif v.T == "STRING":
            return v.V[1:-1]
        elif v.T == "BOOL":
            return v.V.lower() == "true"
        elif v.T == "FLOAT":
            return float(v.V)

    def evaluate_path(self, path):
        return PathEvaluator().evaluate(path, None, self.Current)
        
    def cmp_expression(self, t, path_tree, value, op=None):
        values = self.evaluate_path(path_tree)
        print("ConditionEvaluator.cmp_expression: input values:", values)
        if op == "<":
            values = [v for v in values if isinstance(v, (str, int, float)) and v < value]
        elif op == "<=":
            values = [v for v in values if isinstance(v, (str, int, float)) and v <= value]
        elif op == ">":
            values = [v for v in values if isinstance(v, (str, int, float)) and v > value]
        elif op == ">=":
            values = [v for v in values if isinstance(v, (str, int, float)) and v >= value]
        elif op == "==":
            values = [v for v in values if isinstance(v, (str, int, float, bool)) and v == value]
        n = Node("values", values=values)
        print("                                   returning values:", values)
        return n

In [41]:
# In[67]:


example = '$.track.segments[*] ? (@.HR > 70 && @.location[0] > 47.706)'
#example = "$.track ? (@ < 12)"
parsed = parser.parse(example)
print(parsed.pretty())




condition
  subscript
    dot
      dot
        root
        track
      segments
    index	*
  and_expression
    cmp_expression
      dot
        current
        HR
      >
      value	70
    cmp_expression
      subscript
        dot
          current
          location
        index	0
      >
      value	47.706



In [42]:
converted = PathConverter()(parsed)
print(converted.pretty())



condition m:<lark.tree.Meta object at 0x7fc0c6612668>
+-subscript
  index='*'
  +-dot
    member='segments'
    +-dot
      member='track'
      +-root m:<lark.tree.Meta object at 0x7fc0c6612320>
+-and_expression m:<lark.tree.Meta object at 0x7fc0c6612240>
  +-cmp_expression
    op='>'
    +-dot
      member='HR'
      +-current m:<lark.tree.Meta object at 0x7fc0c6612160>
    +-70
  +-cmp_expression
    op='>'
    +-subscript
      index=0
      +-dot
        member='location'
        +-current m:<lark.tree.Meta object at 0x7fc0c6612470>
    +-47.706


In [43]:
PathEvaluator().evaluate(converted, data)

PathEvaluator.condition: input values: [{'location': [47.763, 13.4034], 'start time': '2018-10-14 10:05:14', 'HR': 73}, {'location': [47.706, 13.2635], 'start time': '2018-10-14 10:39:21', 'HR': 135}]
   evaliating value: {'location': [47.763, 13.4034], 'start time': '2018-10-14 10:05:14', 'HR': 73}
ConditionEvaluator.evaluate(): tree: 140465937508280 
     and_expression m:<lark.tree.Meta object at 0x7fc0c6612240>
    +-cmp_expression
      op='>'
      +-dot
        member='HR'
        +-current m:<lark.tree.Meta object at 0x7fc0c6612160>
      +-70
    +-cmp_expression
      op='>'
      +-subscript
        index=0
        +-dot
          member='location'
          +-current m:<lark.tree.Meta object at 0x7fc0c6612470>
      +-47.706
ConditionEvaluator.current(): returning values: [{'location': [47.763, 13.4034], 'start time': '2018-10-14 10:05:14', 'HR': 73}]
ConditionEvaluator.cmp_expression: input values: Node(values (0 children) (1 data) meta:None)


SyntaxTreeConversionError: Error while processing node condition([Node(values (0 children) (1 data) meta:None), Node(and_expression (2 children) (0 data) meta:<lark.tree.Meta object at 0x7fc0c6612240>)], {}) 